# Narwhals howto

## Motivación
- No queremos tener funciones o libs que estén atadas al tipo de dataframe que usamos, no vamos a replicarla y tener una para polars, otra para padas, modin, cuDF, etc.
- Tampoco pretendemos ir detras de cada nueva lib y andar haciendo migraciones masivas de nuestro código
- Ni queremos obligar a que usen un determinado dataframe, si te sentís bien con pandas, seguí usándolo, yo no te voy a molestar

**¿Qué es?**
Es una capa de abstracción que agregamos a nuestras funciones y nos olvidamos del problema. Veamos un ejemplo concreto para una función que da soporte para dataframe pandas y polars:

In [47]:
import polars as pl
import pandas as pd
import narwhals as nw

In [48]:
print(f"Polars: v{pl.__version__} / Pandas: v{pd.__version__} / Narwhals: v{nw.__version__}")

Polars: v1.10.0 / Pandas: v2.2.3 / Narwhals: v1.18.3


In [40]:
def has_artist_bad(artist, df):
    # para pandas
    if isinstance(df, pd.DataFrame):
        return artist in df["artist"]
    # para polars
    if isinstance(df, (pl.DataFrame, pl.LazyFrame)):
        return artist in df["artist"]

In [43]:
data = {"artist":["charly", "divididos", "sumo", "divididos"],
        "cd":["clics modernos", "amapola del 66", "Llegando los monos", "amapola del 66"], 
        "tema":["Ojos de video tape", "La flor azul", "Estallando desde el oceano", "hombres en U"]}
df_pd = pd.DataFrame(data)
df_pl = pl.DataFrame(data)

In [44]:
has_artist_bad("charly", df_pd)

False

In [45]:
has_artist_bad("charly", df_pl)

True

- Ajá, acá vemos que se comportan diferentes, resulta que el operador in en Pandas busca en los índices de la serie en vez de su contenido, por eso retorna falso. Vamos a arreglarlo:

In [46]:
def has_artist_fixed(artist, df):
    # para pandas
    if isinstance(df, pd.DataFrame):
        return artist in df["artist"].values
    # para polars
    if isinstance(df, (pl.DataFrame, pl.LazyFrame)):
        return artist in df["artist"]

In [30]:
has_artist_fixed("charly", df_pd)

True

In [31]:
has_artist_fixed("charly", df_pl)

True

Es absolutamente indeseable ir modificando el código dependiendo del dataframe en cuestión, hagamos nuestras funciones agnósticas al tipo de dataframe. Es aquí donde **Nawhals** viene a asistirnos.  

In [49]:
def has_artist(artist, df_any):
    # dataframe-agnostic
    df = nw.from_native(df_any)
    return artist in df["artist"]

In [50]:
has_artist("charly", df_pd)

True

In [51]:
has_artist("charly", df_pl)

True